In [11]:
#import necessary libraries
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Embedding
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

## Retrieving Data

In [5]:
#Read csv
train = pd.read_csv (r'train.csv')
test = pd.read_csv (r'test.csv')

In [4]:
df_train = pd.DataFrame(train, columns= ['id','raw_address'])
print(df_train)

            id                                        raw_address
0            0  jl kapuk timur delta sili iii lippo cika 11 a ...
1            1                                 aye, jati sampurna
2            2               setu siung 119 rt 5 1 13880 cipayung
3            3                               toko dita, kertosono
4            4                                      jl. orde baru
...        ...                                                ...
299995  299995               jend ahmad yani 331 kertasari ciamis
299996  299996                 raya cila kko, cilandak timur kel.
299997  299997                     tanjung gusta jl. yaya 2 no 17
299998  299998  jalan cipadu jaya taman asri gang bijaksana 3 ...
299999  299999          jaya maspion permata blok beryl b2  no.58

[300000 rows x 2 columns]


In [6]:
df_test = pd.DataFrame(test, columns= ['id','raw_address'])
print(df_test)

          id                                        raw_address
0          0              s. par 53 sidanegara 4 cilacap tengah
1          1              angg per, baloi indah kel. lubuk baja
2          2                              asma laun, mand imog,
3          3      ud agung rej, raya nga sri wedari karanganyar
4          4                         cut mutia, 35 baiturrahman
...      ...                                                ...
49995  49995                    toko mbak farid semboro semboro
49996  49996     vie - tk. ridho kids, vete 3 cari, 16720 ciawi
49997  49997                mart dan roti bakar malabar, nasio,
49998  49998  graha indah pamulang jl. mujair raya bambu apu...
49999  49999                                               adi,

[50000 rows x 2 columns]


In [7]:
#Call all the raw addresses
df_train.raw_address

0         jl kapuk timur delta sili iii lippo cika 11 a ...
1                                        aye, jati sampurna
2                      setu siung 119 rt 5 1 13880 cipayung
3                                      toko dita, kertosono
4                                             jl. orde baru
                                ...                        
299995                 jend ahmad yani 331 kertasari ciamis
299996                   raya cila kko, cilandak timur kel.
299997                       tanjung gusta jl. yaya 2 no 17
299998    jalan cipadu jaya taman asri gang bijaksana 3 ...
299999            jaya maspion permata blok beryl b2  no.58
Name: raw_address, Length: 300000, dtype: object

In [8]:
#Get a rows of addresses
df_train.loc[2:6]

,id,raw_address
2,2,setu siung 119 rt 5 1 13880 cipayung
3,3,"toko dita, kertosono"
4,4,jl. orde baru
5,5,"raya samb gede, 299 toko bb kids"
6,6,"kem mel raya, no 4 bojong rawalumbu rt 1 36 ra..."


In [9]:
df_train.raw_address.shape

(300000,)

## Tokenization

In [26]:
def tokenize(contents):
  allAddresses = contents.raw_address
  tokens = []
  for address in allAddresses:
    #print(address.split(", "))
    for text in address.split(", "):
      tokens.append(text)
  return tokens

In [29]:
#Get Tokens
train_tokens = tokenize(df_train)

In [31]:
#Number of tokens found
len(train_tokens)

474418

In [33]:
# number of UNIQUE tokens found
len(set(train_tokens))

377844